## Deploy and Discover MCP Server

This notebook demonstrates how to:
1. Deploy an MCP server using the ToolHive operator
2. Connect it to the registry so it's discovered as a `remote` server
3. Verify the server appears in the "default" kubernetes registry
4. Test the server by connecting it to Cursor IDE and running queries

### Prerequisites

- The MCP Registry must be deployed and accessible (from notebook `02-deploy-registry.ipynb`)
- Port-forward the registry API service: `oc port-forward svc/demo-registry-api 8888:8080`


In [ ]:
import json
import requests

REGISTRY_API_SVC_HOST = "localhost"
REGISTRY_API_SVC_PORT = 8888
REGISTRY_PATH = "registry"
EXTENSIONS_PATH = "extension"

def registry_api_url(registry: str, path: str):
    """Build registry API URL"""
    if registry == None or registry == "":
        return f"http://{REGISTRY_API_SVC_HOST}:{REGISTRY_API_SVC_PORT}/{REGISTRY_PATH}/v0.1/{path}"
    return f"http://{REGISTRY_API_SVC_HOST}:{REGISTRY_API_SVC_PORT}/{REGISTRY_PATH}/{registry}/v0.1/{path}"

def get_mcp_servers(registry: str = None):
    """Get the list of MCP servers from the registry API"""
    url = registry_api_url(registry, "servers")
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching servers: {e}")
        raise

def print_server_details(server_item):
    """Print detailed server information"""
    server = server_item["server"]
    print(f"Name:       {server.get('name', 'N/A')}")
    print(f"Description: {server.get('description', 'N/A')}")
    print(f"Version:    {server.get('version', 'N/A')}")
    
    # Check for remotes
    remotes = server.get('remotes', [])
    if remotes:
        print(f"Remotes:    {len(remotes)} remote endpoint(s)")
        for remote in remotes:
            print(f"  - {remote.get('type', 'N/A')}: {remote.get('url', 'N/A')}")
    
    # Package info
    packages = server.get('packages', [])
    if packages:
        package = packages[0]
        print(f"Transport:  {package.get('transport', {}).get('type', 'N/A')}")
        print(f"Identifier: {package.get('identifier', 'N/A')}")
    print()


### Deploy the Kubernetes MCP Server

The manifest includes annotations that connect the server to the registry:

- `toolhive.stacklok.dev/registry-export: "true"` - Enables registry export
- `toolhive.stacklok.dev/registry-url` - Points to the registry API endpoint
- `toolhive.stacklok.dev/registry-description` - Description shown in the registry

In [ ]:
# Deploy the Kubernetes MCP Server
!oc apply -f ../manifests/kubernetes-mcp-server.yaml


In [ ]:
# Wait for the server to be ready
!oc wait mcpserver io-github-stacklok-kubernetes-mcp-server -n toolhive-system --for=condition=Ready --timeout=300s


In [ ]:
# Verify the server pod is running
!oc get pods -n toolhive-system -l app.kubernetes.io/name=io-github-stacklok-kubernetes-mcp-server


### Verify Server Discovery in Registry

Check that the server appears in the "default" kubernetes registry as a remote server.


In [ ]:
# Query the default registry for servers
servers = get_mcp_servers("default")
print(f"Found {servers.get('metadata', {}).get('count', 0)} server(s) in default registry\n")

for server_item in servers.get("servers", []):
    print("=" * 80)
    print_server_details(server_item)


### Port-Forward the Proxy Service

To connect the server to Cursor IDE, we need to expose the proxy service locally.


In [ ]:
# Get the proxy service name
!oc get svc -n toolhive-system -l app.kubernetes.io/name=io-github-stacklok-kubernetes-mcp-server -o jsonpath='{.items[0].metadata.name}'


**Note**: Run the following command in a separate terminal to port-forward the proxy service:

```bash
oc port-forward svc/io-github-stacklok-kubernetes-mcp-server-proxy -n toolhive-system 8080:8080
```

This will expose the proxy service on `localhost:8080`.


### Connect to Cursor IDE

1. Open Cursor IDE settings
2. Navigate to **Features** → **Model Context Protocol**
3. Click **Add Server** or edit your MCP settings
4. Add a new server configuration:

```json
{
  "mcpServers": {
    "kubernetes": {
      "url": "http://localhost:8080",
      "transport": "sse"
    }
  }
}
```

5. Save the configuration and restart Cursor IDE if needed
6. The Kubernetes MCP server should now be available in your AI agent


### Test the Server

Once connected to Cursor IDE, you can test the server by asking the AI agent to:

**Example queries:**
- "List all namespaces in the cluster"
- "Show me pods in the toolhive-system namespace"
- "What resources are available in the current namespace?"
- "Describe the demo-registry-api pod"

The AI agent will use the Kubernetes MCP server to query your OpenShift cluster and return the results.


### Verify Server Tools

You can also verify the server is working by checking its available tools:


In [ ]:
# Test the proxy endpoint (requires port-forward to be running)
try:
    # Note: This requires the port-forward to be active
    response = requests.get("http://localhost:8080/health", timeout=5)
    response.raise_for_status()
    print(f"Server health check: {response.status_code}")
    print("✅ Server proxy is accessible")
except requests.exceptions.ConnectionError:
    print("⚠️  Could not connect to server. Make sure port-forward is running:")
    print("   oc port-forward svc/io-github-stacklok-kubernetes-mcp-server-proxy -n toolhive-system 8080:8080")
except requests.exceptions.HTTPError as e:
    print(f"⚠️  Server returned error: {e}")
except Exception as e:
    print(f"Error: {e}")


## Conclusion

We successfully:

- ✅ Deployed the Kubernetes MCP Server using the ToolHive operator
- ✅ Connected it to the registry with proper annotations
- ✅ Verified it appears as a remote server in the "default" kubernetes registry
- ✅ Set up port-forwarding for local access
- ✅ Configured Cursor IDE to connect to the server

The Kubernetes MCP server is now ready to use with your AI agent in Cursor IDE. You can query Kubernetes resources, manage deployments, and interact with your OpenShift cluster through natural language queries.
